# Test noise_sim

In [1]:
from noise_sim_tc import noise_sim
import tensorcircuit as tc
from tensorcircuit.channels import amplitudedampingchannel
import numpy as np

# 从 txt 文件读取电路

qubit_num = 6 # 我们读取一个 6 qubit 的 Ising model 的演化电路. 

with open("openqasm_str.txt", "r", encoding="utf-8") as f:
    openqasm_str = f.read()

print("电路: ", openqasm_str)

# 转换为 tc 格式的电路并且输出
c_mat = tc.densitymatrix.DMCircuit.from_openqasm(openqasm_str)
print("noise-free circuit expectation:", c_mat.expectation_ps(z=[windex for windex in range(qubit_num)]))
c_mat.draw()

c_mat_noise = noise_sim(c_mat)
print("noiseless circuit expectation:", c_mat.expectation_ps(z=[windex for windex in range(qubit_num)]))
print("noisy circuit expectation:", c_mat_noise.expectation_ps(z=[windex for windex in range(qubit_num)]))

d:\Anaconda\envs\tc-tf\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


电路:  OPENQASM 2.0;
include "qelib1.inc";
qreg q[6];
h q[1];
cz q[0],q[1];
h q[1];
rz(-1.5707963267948966) q[1];
h q[1];
cz q[0],q[1];
h q[1];
h q[2];
cz q[1],q[2];
h q[2];
rz(-1.5707963267948966) q[2];
h q[2];
cz q[1],q[2];
h q[2];
h q[3];
cz q[2],q[3];
h q[3];
rz(-1.5707963267948966) q[3];
h q[3];
cz q[2],q[3];
h q[3];
h q[4];
cz q[3],q[4];
h q[4];
rz(-1.5707963267948966) q[4];
h q[4];
cz q[3],q[4];
h q[4];
h q[5];
cz q[4],q[5];
h q[5];
rz(-1.5707963267948966) q[5];
h q[5];
cz q[4],q[5];
h q[5];
h q[0];
cz q[5],q[0];
h q[0];
rz(-1.5707963267948966) q[0];
h q[0];
cz q[5],q[0];
h q[0];
rx(0.1) q[0];
rx(0.1) q[1];
rx(0.1) q[2];
rx(0.1) q[3];
rx(0.1) q[4];
rx(0.1) q[5];
h q[1];
cz q[0],q[1];
h q[1];
rz(-1.5707963267948966) q[1];
h q[1];
cz q[0],q[1];
h q[1];
h q[2];
cz q[1],q[2];
h q[2];
rz(-1.5707963267948966) q[2];
h q[2];
cz q[1],q[2];
h q[2];
h q[3];
cz q[2],q[3];
h q[3];
rz(-1.5707963267948966) q[3];
h q[3];
cz q[2],q[3];
h q[3];
h q[4];
cz q[3],q[4];
h q[4];
rz(-1.5707963267948966) 

In [2]:
circuit = tc.Circuit(2)
circuit.cz(0, 1)
circuit_noise = noise_sim(circuit)
print("noiseless circuit expectation:", circuit.expectation_ps(z=[0,1]))
print("noisy circuit expectation:", circuit_noise.expectation_ps(z=[0,1]))

noiseless circuit expectation: (1+0j)
noisy circuit expectation: (1+0j)


# Functions

In [3]:
import numpy as np
from itertools import product

# 单比特 Pauli
I = np.array([[1,0],[0,1]], dtype=complex)
X = np.array([[0,1],[1,0]], dtype=complex)
Y = np.array([[0,-1j],[1j,0]], dtype=complex)
Z = np.array([[1,0],[0,-1]], dtype=complex)
paulis = [I, X, Y, Z]
labels = ['I', 'X', 'Y', 'Z']

# CZ gate
CZ = np.diag([1,1,1,-1]).astype(complex)

def pauli_word(a, b):
    return np.kron(paulis[a], paulis[b])

# === 预计算映射表 ===
mapping = {}
for c, d in product(range(4), repeat=2):  # 遍历 Pm
    Pm = pauli_word(c, d)
    Pn_mat = CZ @ Pm @ CZ
    for a, b in product(range(4), repeat=2):  # 遍历 Pn
        Pw = pauli_word(a, b)
        if np.allclose(Pn_mat, Pw):
            mapping[(a, b)] = (c, d, +1)
        elif np.allclose(Pn_mat, -Pw):
            mapping[(a, b)] = (c, d, -1)

def find_m_for_given_n(a, b):
    """O(1) 查表"""
    return mapping.get((a, b))

# === 示例 ===
a, b = 3, 2  # n = X ⊗ Y
result = find_m_for_given_n(a, b)
c, d, phase = result
print(f"n = {labels[a]}⊗{labels[b]}, m = {labels[c]}⊗{labels[d]}, phase={phase}")
print("CZ0,1+ P_n CZ0,1 = Pm")
print(f"CZ0,1+ {labels[a]}⊗{labels[b]} CZ0,1 = {labels[c]}⊗{labels[d]}")
Pn=pauli_word(a, b)
Pm=pauli_word(c, d)
print(CZ @ Pn @ CZ == phase*Pm)


n = Z⊗Y, m = I⊗Y, phase=1
CZ0,1+ P_n CZ0,1 = Pm
CZ0,1+ Z⊗Y CZ0,1 = I⊗Y
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


In [ ]:
from noise_sim_tc import noise_sim
import tensorcircuit as tc
import numpy as np

def cz_fidelity(observable, n_cz, isNoisy=True):
    """
    observable: list, e.g. [0,3]，0,1,2,3分别代表I,X,Y,Z，顺序对应比特0和1
    n_cz: int, CZ门数量
    返回有噪声电路的期望值
    """
    # 泡利算符映射
    pauli_keys = ['i', 'x', 'y', 'z']
    obs_dict = {pauli_keys[observable[0]]: [0], pauli_keys[observable[1]]: [1]}

    # 生成泡利算符列表
    pauli_dict = {
        'i': np.array([[1, 0], [0, 1]]),
        'x': np.array([[0, 1], [1, 0]]),
        'y': np.array([[0, -1j], [1j, 0]]),
        'z': np.array([[1, 0], [0, -1]])
    }
    # 生成泡利门列表
    pauli_gate_funcs = [lambda c, q: None,  # I门不操作
                lambda c, q: c.x(q),
                lambda c, q: c.y(q),
                lambda c, q: c.z(q)]
    # 构造特征向量
    def get_eigenvector(idx):
        if idx == 0:
            return np.array([1, 0])  # I本征态任意，取|0>
        elif idx == 1:
            return np.array([1, 1]) / np.sqrt(2)  # X本征态
        elif idx == 2:
            return np.array([1, 1j]) / np.sqrt(2)  # Y本征态
        elif idx == 3:
            return np.array([1, 0])  # Z本征态
    state_list = [get_eigenvector(observable[0]), get_eigenvector(observable[1])]
    init_state = np.kron(state_list[0], state_list[1])
    # init_state = 
    

    # ================================================================================================
    # # （有 噪声前后的泡利算符）
    #     # CZ 门和带有噪声模拟的CZ门
    #     circuit_cz= tc.Circuit(2)
    #     circuit_cz.cz(0, 1)
    #     circuit_noise = noise_sim(circuit_cz)

    # # 构造电路
    # expectation_sum = 0
    # for i in range(4):
    #     for j in range(4):
    #         # 构建门
    #         circuit = tc.Circuit(2, inputs=init_state)

    #         circuit_before_channel = tc.Circuit(2)
    #         pauli_gate_funcs[i](circuit_before_channel, 0)
    #         pauli_gate_funcs[j](circuit_before_channel, 1)

    #         circuit_after_channel = tc.Circuit(2)
    #         cz_i, cz_j, phase = find_m_for_given_n(i, j)
    #         pauli_gate_funcs[cz_i](circuit_after_channel, 0)
    #         pauli_gate_funcs[cz_j](circuit_after_channel, 1)
            
    #         # 构建电路
    #         for _ in range(n_cz):
    #             # 添加泡利算符(噪声前)
    #             circuit.append(circuit_before_channel)
    #             # 添加CZ门或者带有噪声模拟的CZ门
    #             if isNoisy:
    #                 circuit.append(circuit_noise)
    #             else:
    #                 circuit.append(circuit_cz)
    #             # 添加泡利算符(噪声后)
    #             circuit.append(circuit_after_channel)
                
    #         # 计算期望值
    #         expectation_sum+=circuit.expectation_ps(**obs_dict) 
    # return expectation_sum / (4**2) 
    

    # ================================================================================================
    # （无 噪声前后的泡利算符）
    # 构造电路
    # circuit = tc.Circuit(2, inputs=init_state)
    circuit = tc.densitymatrix.DMCircuit(2)#,inputs=init_state)
    for _ in range(n_cz):
        # circuit_cz = tc.Circuit(2)
        circuit.cz(0, 1)
        # circuit_noise = noise_sim(circuit_cz)
        # if isNoisy:
        #     circuit.append(circuit_noise)
        # else:
        #     circuit.append(circuit_cz)
    circuit = noise_sim(circuit) if isNoisy else circuit
    # 计算期望值
    expectation = circuit.expectation_ps(**obs_dict)
    return expectation

# Get fidelities

In [11]:
observable=[2,1]
result = cz_fidelity(observable, n_cz=2,isNoisy=False)
result_1 = cz_fidelity(observable, n_cz=2,isNoisy=True)
result_2 = cz_fidelity(observable, n_cz=4,isNoisy=True)
result_3 = cz_fidelity(observable, n_cz=20,isNoisy=True)
print(result, result_2, result_2, result_3)

0j 0j 0j 0j


In [6]:
f_noiseless=[]
for i in range(4):
    for j in range(4):
        fidelity2_ij = cz_fidelity([i,j], n_cz=2, isNoisy=False)
        fidelity4_ij = cz_fidelity([i,j], n_cz=4, isNoisy=False)
        if fidelity2_ij == 0:
            f_noiseless.append(0)
        else:
            f_noiseless.append(fidelity4_ij / fidelity2_ij)
print(f_noiseless)

[(1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j), (1+0j)]


In [7]:
f_noisy=[]
for i in range(4):
    for j in range(4):
        fidelity2_ij = cz_fidelity([i,j], n_cz=2, isNoisy=True)
        fidelity4_ij = cz_fidelity([i,j], n_cz=4, isNoisy=True)
        if fidelity2_ij == 0:
            f_noisy.append(0)
        else:
            f_noisy.append(fidelity4_ij / fidelity2_ij)
print(f_noisy)



[(1+0j), 0, 0, (1+0j), 0, 0, 0, 0, 0, 0, 0, 0, (1+0j), 0, 0, (1+0j)]
